In [43]:
from bs4 import BeautifulSoup
import httpx
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os

In [47]:
query_images = ['cachorro', 'gato']

In [48]:
def imgScraper(query_images):
    driver = webdriver.Chrome()  

    url = f"https://unsplash.com/pt-br/s/fotografias/{query_images}"
    driver.get(url)

    scroll_pause_time = 3  
    screen_height = driver.execute_script("return window.screen.height;")
    load_more = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[1]/div/div[2]/div[4]/div/div[3]/div[1]/button')
    load_more.click()
    i = 1

    while True:
        driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
        i += 1
        time.sleep(scroll_pause_time)

        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        if screen_height*i > scroll_height: # ALTERANDO ESSA LINHA PARA "i > (numero de scrolls desejado)" É POSSÍVEL EVITAR O SCROLL INFINITO
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    return soup

def imgDownloader(soup_images, query_images):
    div = soup_images.find_all('div', attrs={"id":":R1clltp:"})
    img = div[0].find_all('img', attrs={"itemprop":"thumbnailUrl"})
    k=0
    for i in img:
        if ('https' in i['src']) and ('profile' not in i['src']):
            with open(f"./dataset/{query_images}/{query_images}{k+1}.png", "wb") as file:
                image = httpx.get(i['src'])
                file.write(image.content)
            k += 1

In [49]:
query_images = ['cachorro', 'gato']
current_dir = os.getcwd()

for i_images in range(len(query_images)):
    new_dir = current_dir + '\\dataset' + '\\' + query_images[i_images]
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    soup_images = imgScraper(query_images[i_images])
    imgDownloader(soup_images, query_images[i_images])